### First few cells is just data preparation

My implementation is build on top of nequip and allegro python libraries to simplify data load and preprocessing.
The wigner simbols are from e3nn
Here are the links
https://github.com/mir-group/nequip
https://github.com/mir-group/allegro

In [1]:
# reference example
from nequip.data import dataset_from_config
from nequip.utils import Config
#from nequip.utils.misc import get_default_device_name
#from nequip.utils.config import _GLOBAL_ALL_ASKED_FOR_KEYS

from nequip.model import model_from_config


default_config = dict(
    root="./",
    tensorboard=False,
    wandb=False,
    model_builders=[
        "SimpleIrrepsConfig",
        "EnergyModel",
        "PerSpeciesRescale",
        "StressForceOutput",
        "RescaleEnergyEtc",
    ],
    dataset_statistics_stride=1,
    device='cpu',
    default_dtype="float64",
    model_dtype="float32",
    allow_tf32=True,
    verbose="INFO",
    model_debug_mode=False,
    equivariance_test=False,
    grad_anomaly_mode=False,
    gpu_oom_offload=False,
    append=False,
    warn_unused=False,
    _jit_bailout_depth=2,  # avoid 20 iters of pain, see https://github.com/pytorch/pytorch/issues/52286
    # Quote from eelison in PyTorch slack:
    # https://pytorch.slack.com/archives/CDZD1FANA/p1644259272007529?thread_ts=1644064449.039479&cid=CDZD1FANA
    # > Right now the default behavior is to specialize twice on static shapes and then on dynamic shapes.
    # > To reduce warmup time you can do something like setFusionStrartegy({{FusionBehavior::DYNAMIC, 3}})
    # > ... Although we would wouldn't really expect to recompile a dynamic shape fusion in a model,
    # > provided broadcasting patterns remain fixed
    # We default to DYNAMIC alone because the number of edges is always dynamic,
    # even if the number of atoms is fixed:
    _jit_fusion_strategy=[("DYNAMIC", 3)],
    # Due to what appear to be ongoing bugs with nvFuser, we default to NNC (fuser1) for now:
    # TODO: still default to NNC on CPU regardless even if change this for GPU
    # TODO: default for ROCm?
    _jit_fuser="fuser1",
)

# All default_config keys are valid / requested
#_GLOBAL_ALL_ASKED_FOR_KEYS.update(default_config.keys())

In [2]:
config = Config.from_file('./configs/example_ETN.yaml', defaults=default_config)
    

dataset = dataset_from_config(config, prefix="dataset")

validation_dataset = None

dataset[0]

AtomicData(atom_types=[21, 1], cell=[3, 3], edge_cell_shift=[364, 3], edge_index=[2, 364], forces=[21, 3], pbc=[3], pos=[21, 3], total_energy=[1])

In [3]:
# Trainer
from nequip.train.trainer import Trainer
from e3nn import o3

trainer = Trainer(model=None, **Config.as_dict(config))

# what is this
# to update wandb data?
config.update(trainer.params)

# = Train/test split =
trainer.set_dataset(dataset, validation_dataset)

# Some hyperparameteres
Nc = 10 # number of chennels for F features from ETN paper
N_rank_spec = 4 # hidden rank of reduction for type radial tensor
config['Nc'] = Nc
config['N_rank_spec'] = N_rank_spec

# ETN parameters
config['d'] = 4 # dimention of the tensor train
config['N_rank_ett'] = [4, 4, 4] # ranks of tensor train


# = Build model =
final_model = model_from_config(
    config=config, initialize=True, dataset=trainer.dataset_train
)

DEBUG:root:* Initialize Output
  ...generate file name results/aspirin/example/log
  ...open log file results/aspirin/example/log
  ...generate file name results/aspirin/example/metrics_epoch.csv
  ...open log file results/aspirin/example/metrics_epoch.csv
  ...generate file name results/aspirin/example/metrics_initialization.csv
  ...open log file results/aspirin/example/metrics_initialization.csv
  ...generate file name results/aspirin/example/metrics_batch_train.csv
  ...open log file results/aspirin/example/metrics_batch_train.csv
  ...generate file name results/aspirin/example/metrics_batch_val.csv
  ...open log file results/aspirin/example/metrics_batch_val.csv
  ...generate file name results/aspirin/example/best_model.pth
  ...generate file name results/aspirin/example/last_model.pth
  ...generate file name results/aspirin/example/trainer.pth
  ...generate file name results/aspirin/example/config.yaml
Torch device: cpu
instantiate Loss
...Loss_param = dict(
...   optional_args =

Replace string dataset_forces_rms to 31.252248764038086
Initially outputs are globally scaled by: 31.252248764038086, total_energy are globally shifted by None.
PerSpeciesScaleShift's arguments were in dataset units; rescaling:
  Original scales: [H: 31.252249, C: 31.252249, O: 31.252249] shifts: [H: -19318.355469, C: -19318.355469, O: -19318.355469]
  New scales: [H: 1.000000, C: 1.000000, O: 1.000000] shifts: [H: -618.142883, C: -618.142883, O: -618.142883]


In [4]:
import torch
from torch.nn.functional import one_hot
from nequip.data import AtomicData, AtomicDataDict
from torch.nn.functional import one_hot
from e3nn.nn import FullyConnectedNet
from allegro import with_edge_spin_length
from allegro import _keys
from torch import nn
import math

trainer.model = final_model

# Test configuration stores as dict of parameters
data0 = AtomicData.to_AtomicDataDict(dataset[0])

In [5]:
import torch
from torch.nn.functional import one_hot
from nequip.data import AtomicData, AtomicDataDict
from torch.nn.functional import one_hot
from e3nn.nn import FullyConnectedNet
    
from torch import nn
import math

# forward pass
data_new = final_model(data0)

### Main module

I'll skip the implementation of feature vector F because it heavily relies on nequip code for neighbor lists operations.
If you want you can look into the following files or ask me to add it to the notebook.

Otherwise look into this file

allegro/modules/ETN.py 

and corresponding layers


In [6]:
from typing import Optional, List
import math
import functools

import torch
from torch import nn

from e3nn import o3
from e3nn.util.jit import compile_mode

from nequip.data import AtomicDataDict # dict of base keys
from nequip.nn import GraphModuleMixin # base class for GNN (stores irreps in and out)

from allegro import _keys # just dict of additional keys

from e3nn.o3 import wigner_3j # wigner_3j matrixes for three spherical harmonics coupling



# Triangular ineguality for path existance
def tri_ineq(l1, l2, l3):
    return max([l1, l2, l3]) <= min([l1 + l2, l2 + l3, l1 + l3])


@compile_mode("script")
class ETN_Module(nn.Module, GraphModuleMixin):
    def __init__(self,
                 d: int,
                 N_rank_ett: List[int], 
                 irreps_in=None,
                 out_field: str = AtomicDataDict.PER_ATOM_ENERGY_KEY):
        
        super().__init__()
        self.out_field = out_field
        
        
        self.d = d
        self.Nc = irreps_in[_keys.NODE_FEATURES_F][0][0]
        self.register_buffer("N_rank_ett", torch.as_tensor(N_rank_ett, dtype=torch.long))
        
        # set up irreps
        self._init_irreps(
            irreps_in=irreps_in,
            required_irreps_in=[
                _keys.NODE_FEATURES_F
            ],
            irreps_out={_keys.NODE_FEATURES_ETN: o3.Irreps(
                    [(self.Nc, ir) for _, ir in irreps_in[_keys.NODE_FEATURES_F] ]),
                        out_field: o3.Irreps([(1, (0, 1))])}
        )
        
        
        # Parameters of the network
        
        # tensors for atomic features encoding
        lmax = irreps_in[_keys.EDGE_FEATURES_F].lmax # maximum spherical harmonic
        self.lmax = lmax
        
        # Second order cores(first and last)
        self.core2_1 = torch.nn.Parameter(torch.Tensor(lmax+1, self.Nc, N_rank_ett[0]))
        self.core2_d = torch.nn.Parameter(torch.Tensor(lmax+1, N_rank_ett[-1], self.Nc))
        
        
        
        # Third order cores
        
        # all wigner3j 
        self.w3j_big = [[[wigner_3j(l1, l2, l3) if tri_ineq(l1, l2, l3) else None for l3 in range (lmax+1)] for l2 in range(lmax+1)] for l1 in range(lmax+1)]
        
        # all third order free parameters
        self.cores3 = [{(l1, l2, l3): torch.nn.Parameter(torch.Tensor(N_rank_ett[r], self.Nc, N_rank_ett[r+1])) for l1 in range (lmax+1) for l2 in range(lmax+1) for l3 in range(lmax+1) if tri_ineq(l1, l2, l3)} for r in range(d - 2)] 
        
        self.reset_parameters()
        
    def forward(self, data: AtomicDataDict.Type) -> AtomicDataDict.Type:
        
        # Input features
        F = data[_keys.NODE_FEATURES_F]
        
        # Defining tensors for TorchScript
        u_out = torch.zeros((F.shape[0], F.shape[1], self.N_rank_ett[-1]), dtype=F.dtype,
            device=F.device) # temporary verctor output of etn
            
        
        data[_keys.NODE_FEATURES_ETN] = torch.zeros_like(F, dtype=F.dtype,
            device=F.device) # final feature output
        
        slices = self.irreps_in[AtomicDataDict.EDGE_ATTRS_KEY].slices() # slices over irreps
        
        # First transform using second order tensors
        for i, slice in enumerate(slices):
            u_out[:, slice, :] = torch.einsum('ij,Nmj->Nmi', self.core2_d[i], F[:, slice, :])
        
        # Series third order tensors
        for i in range(self.d - 2 - 1, -1, -1):
            
            # TODO: now define localy, mb define for all, to ensure computational graph
            T_2_tmp = [[torch.zeros(F.shape[0], 2*l1+1, 2*l2+1, self.N_rank_ett[i - 1], self.Nc, dtype=F.dtype, device=F.device) for l2 in range(self.lmax + 1)] for l1 in range(self.lmax + 1)] # result of first reduction of order 3 tensor
            
            # First contraction with previous feature vector
            for l1 in range(self.lmax + 1):
                for l2 in range(self.lmax + 1):
                    for l3, slice in enumerate(slices):
                        if tri_ineq(l1, l2, l3):
                            T_3 = self.w3j_big[l1][l2][l3][..., None, None, None] * self.cores3[i][(l1, l2, l3)][None, None, None, ...]
                            T_2_tmp[l1][l2] += torch.einsum('abcijk,Nck->Nabij', T_3, u_out[:, slice, :])
            
            
            # Second contraction with F vector
            u_out_new = torch.zeros((F.shape[0], F.shape[1], self.N_rank_ett[i - 1]), dtype=F.dtype,
                            device=F.device) # temporary verctor output of etn
            
            for l1 in range(self.lmax + 1):    
                for l2, slice in enumerate(slices):
                    u_out_new[:, slices[l1], :] += torch.einsum('Nabij,Nbj->Nai', T_2_tmp[l1][l2], F[:, slice, :])
            
            
            u_out = u_out_new
            
        # Last transform using second order tensor
        for i, slice in enumerate(slices):
            data[_keys.NODE_FEATURES_ETN][:, slice, :] = torch.einsum('ij,Nmj->Nmi', self.core2_1[i], u_out[:, slice, :])
        
        
        # Reduction to scalar
        data[self.out_field] = ( data[_keys.NODE_FEATURES_ETN] * F ).sum(dim = (-2, -1))
        

        return data
    
    def reset_parameters(self):
        torch.nn.init.kaiming_uniform_(self.core2_1, a=math.sqrt(3))
        torch.nn.init.kaiming_uniform_(self.core2_d, a=math.sqrt(3))
        
        for core in self.cores3:
            for key in core:
                torch.nn.init.kaiming_uniform_(core[key], a=math.sqrt(3))

### Usage Example

In [7]:
ETN = ETN_Module(d = config['d'],
                         N_rank_ett = config['N_rank_ett'], 
                         irreps_in = final_model.irreps_out,
                         out_field = AtomicDataDict.PER_ATOM_ENERGY_KEY)

In [8]:
data_new_new = ETN(data_new)

### Some equivariance testing of the final model

In [9]:

import torch
from torch.nn.functional import one_hot
from nequip.data import AtomicData, AtomicDataDict
from torch.nn.functional import one_hot
from e3nn.nn import FullyConnectedNet
from allegro import with_edge_spin_length
from allegro import _keys
from torch import nn
import math

data = data0


import copy

data_rot = {key: torch.clone(data0[key]) for key in data0}

irreps_sh = o3.Irreps('1x0e + 1x1o + 1x2e') #o3.Irreps.spherical_harmonics(lmax=2)
irreps_sh_r = o3.Irreps('1x1o')

alpha, beta, gamma = o3.rand_angles(100)

rot_matrix = irreps_sh.D_from_angles(alpha[0], beta[0], gamma[0])
rot_matrix_r = irreps_sh_r.D_from_angles(alpha[0], beta[0], gamma[0])


data_rot['pos'] = data_rot['pos'] @ rot_matrix_r

In [10]:
torch.manual_seed(32)

data_out = final_model(data)

data_out_rot =final_model(data_rot)


F = data_out['node_features_F'] # features
F_rot =data_out_rot['node_features_F'] # features from rotated positions
F_rot_rot = torch.einsum('Njn,jk->Nkn', F_rot, rot_matrix.T) # rotated features from rotated positions

assert torch.allclose(F, F_rot_rot, atol=1e-05)
print('F is equivariant')
    
    
ETN_out = data_out['node_features_ETN'] # ETN out features
ETN_out_rot =data_out_rot['node_features_ETN'] # ETN out features from rotated positions
ETN_out_rot_rot = torch.einsum('Njn,jk->Nkn', ETN_out_rot, rot_matrix.T) # rotated ETN out features from rotated positions

assert torch.allclose(ETN_out, ETN_out_rot_rot, atol=1e-05)
print('ETN forward output is equivariant')

at_en = data_out['atomic_energy'] # atomic energy
at_en_rot = data_out_rot['atomic_energy'] # atomic energy from rotated positions

assert torch.allclose(at_en, at_en_rot, atol=1e-05)
print('atomic energy is invariant')

tot_en = data_out['total_energy'] # atomic energy
tot_en_rot = data_out_rot['total_energy'] # atomic energy from rotated positions

assert torch.allclose(tot_en, tot_en_rot, atol=1e-05)
print('total energy is invariant')

F is equivariant
ETN forward output is equivariant
atomic energy is invariant
total energy is invariant


In [13]:
f_out = data_out['forces'] # ETN out features
f_out_rot =data_out_rot['forces'] # ETN out features from rotated positions
#f_out_rot_rot = torch.einsum('Njn,jk->Nkn', f_out_rot, rot_matrix_r.T) # rotated ETN out features from rotated positions

f_out_rot_rot = f_out_rot @ rot_matrix_r.T

assert torch.allclose(f_out, f_out_rot_rot, atol=1e-05)
print('ETN forward output is equivariant')

ETN forward output is equivariant
